In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:



import os
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
from huggingface_hub import login
import torch
from torch.utils.data import Dataset, DataLoader
import wandb

# Set your key

wandb.login(key='Replace_Key') # Weights & Biases (wandb) for experiment tracking and uses environment variables for API key management

API_KEY = "Replace_Key"  # Replace with a temporary key for Colab
os.environ["API_KEY"] = API_KEY  # Set environment variable for the session

# Optionally log in to Hugging Face (if not done globally)
# login()

# Load the MultiWOZ v22 dataset and select a sample (e.g., 100 samples)
dataset = load_dataset("multi_woz_v22")

# Select a smaller sample for testing (e.g., 100 samples)
sample_size = 100
train_sample = dataset['train'].select(range(sample_size))
valid_sample = dataset['validation'].select(range(20))  # Select 20 samples for validation

# Print an example dialogue to see the format
print(train_sample[0])  # View the first conversation

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('microsoft/DialoGPT-small')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Define the preprocessing function
def preprocess_function(examples):
    """
    Preprocesses and tokenizes the conversation data.
    Turns each dialogue in the multi-woz dataset into a "Customer: ... Support: ..." format.

    Args:
        examples: A dictionary containing dialogue information.

    Returns:
        A dictionary containing tokenized input IDs, attention masks, and labels.
    """
    # We assume that 'turns' contains the conversation turns with the speaker and utterance
    concatenated_examples = []

    for dialogue in examples['turns']:
        dialogue_text = []
        for speaker, utterance in zip(dialogue['speaker'], dialogue['utterance']):
            if speaker == 0:  # User (0 is usually the user in this dataset)
                dialogue_text.append(f"Customer: {utterance}")
            else:  # Agent (1 is usually the agent in this dataset)
                dialogue_text.append(f"Support: {utterance}")

        # Join all the turns for the conversation into one string
        concatenated_examples.append(" ".join(dialogue_text))

    return tokenizer(concatenated_examples, truncation=True, padding="max_length", max_length=512, return_tensors="pt")

# Preprocess the sample dataset
tokenized_inputs = train_sample.map(preprocess_function, batched=True, remove_columns=train_sample.column_names)
valid_tokenized_inputs = valid_sample.map(preprocess_function, batched=True, remove_columns=valid_sample.column_names)

# Creating a validation set from the training dataset (90% for training, 10% for validation)
train_size = int(0.9 * len(tokenized_inputs))  # 90% for training
train_dataset = tokenized_inputs.select(range(train_size))  # First 90%
valid_dataset = tokenized_inputs.select(range(train_size, len(tokenized_inputs)))  # Last 10%

class ConversationDataset(Dataset):
    """
    Custom dataset class to handle tokenized conversation data.
    """
    def __init__(self, tokenized_data):
        self.input_ids = tokenized_data['input_ids']
        self.attention_mask = tokenized_data['attention_mask']

    def __getitem__(self, idx):
        """
        Returns a data sample for a given index.
        """
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.input_ids[idx]  # Labels are the same as input_ids for language modeling
        }

    def __len__(self):
        """
        Returns the total number of samples in the dataset.
        """
        return len(self.input_ids)

# Create dataset objects for training and validation
train_data = ConversationDataset(train_dataset)
valid_data = ConversationDataset(valid_dataset)

# Load the pre-trained model (DialoGPT-small)
model = GPT2LMHeadModel.from_pretrained('microsoft/DialoGPT-small')

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory for checkpoints and models
    num_train_epochs=3,              # Number of training epochs
    per_device_train_batch_size=2,   # Batch size per device during training
    per_device_eval_batch_size=2,    # Batch size per device during evaluation
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,                # Log every 10 steps
    evaluation_strategy="epoch",     # Evaluate at the end of each epoch
)

# Create the Trainer
trainer = Trainer(
    model=model,                         # The pre-trained model to fine-tune
    args=training_args,                  # Training arguments
    train_dataset=train_data,            # Training dataset
    eval_dataset=valid_data             # Validation dataset
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_pretrained("./customer_support_chatbot")
tokenizer.save_pretrained("./customer_support_chatbot")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anvcse2007 (anvcse2007-university-of-illinois-urbana-champaign) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

multi_woz_v22.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,2.146600,1.147588
2,1.854000,1.006987
3,1.257300,0.975803


AttributeError: 'Trainer' object has no attribute 'save_pretrained'

In [3]:
model.save_pretrained("./customer_support_chatbot")
tokenizer.save_pretrained("./customer_support_chatbot")

('./customer_support_chatbot/tokenizer_config.json',
 './customer_support_chatbot/special_tokens_map.json',
 './customer_support_chatbot/vocab.json',
 './customer_support_chatbot/merges.txt',
 './customer_support_chatbot/added_tokens.json')

In [5]:

from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the tokenizer and model
model_path = './customer_support_chatbot'  # Path to your fine-tuned model
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Set the model to evaluation mode
model.eval()

# Function to generate text based on a prompt
def generate_response(prompt, max_length=50):
    """
    Generates a response based on the given prompt using the fine-tuned chatbot model.

    Args:
        prompt: The input prompt for the chatbot.
        max_length: The maximum length of the generated response.

    Returns:
        The generated response text.
    """
    # Encode the prompt and create attention mask
    inputs = tokenizer(prompt, return_tensors='pt', padding=True)
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    # Generate output
    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask, # Pass the attention mask here
        max_length=max_length + len(input_ids[0]),
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        repetition_penalty=2.5,
        top_p=0.92,
        temperature=0.85,
        do_sample=True,
        top_k=125,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and return the output text
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Example usage for customer support
prompt = "Customer: I'm having trouble logging into my account. Support:"
response = generate_response(prompt)
print(response)

prompt = "Customer: My order hasn't arrived yet. Support:"
response = generate_response(prompt)
print(response)

Customer: I'm having trouble logging into my account. Support: There is no problem with the password and address information? Customer : It has been successful, thank you very much for your help!
Customer: My order hasn't arrived yet. Support: I'd like to know the address of your nearest restaurant please! Customer : Does it have free parking for 3 people and a car? Is there any type or price range you would recommend seeing on that day, in particular? No entry fee. Phone
